# Limpeza da base de dados Viagens_2025

Importando as bibliotecas para iniciar a limepeza

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

Criando as variáveis de ambiente da URL do banco de dados e criando uma função para que as linhas que forem descartadas sejam guardadas

In [2]:
DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL)

linhas_descartadas = []

def capturar_linha_ruim(linha):
    linhas_descartadas.append(linha)
    return None

Leitura do csv que será base do banco de dados

In [3]:
df = pd.read_csv(
    "../Data Layer/raw/dados_brutos.csv",
    sep=";",
    encoding="latin1",
    na_values=["Sem informação", -11, -1],
    engine="python",
    on_bad_lines=capturar_linha_ruim
)


Limpeza das colunas para que elas tenham a formatação padrão de UTF-8

In [4]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.normalize('NFKD')
      .str.encode('ascii', errors='ignore')
      .str.decode('utf-8')
      .str.replace(' ', '_')
      .str.replace('-', '_')
)

Isolando a justificativa de viagem para que todas as partes vazias sejam substituídas por "Sem informação"

In [5]:
mask = df['justificativa_urgencia_viagem'].isna()
df.loc[mask, 'justificativa_urgencia_viagem'] = 'Sem informação'

Cria uma variável para salvar as colunas financeiras

In [6]:
colunas_financeiras = [
    'valor_diarias',
    'valor_passagens',
    'valor_devolucao',
    'valor_outros_gastos'
]

Formata as colunas financeiras para que seja composta apenas por números FLOAT para que se possam fazer calculos com os custos

In [7]:
for col in colunas_financeiras:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace('R$', '', regex=False)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

df[colunas_financeiras] = df[colunas_financeiras].astype(float)


Calcula os gastos totais de cada viagem desconsiderando o valor devolvido pelo viajante

In [8]:
df['tot_gas'] = (
    df['valor_diarias'] +
    df['valor_passagens'] +
    df['valor_outros_gastos'] -
    df['valor_devolucao']
).round(2)

Transforma todas as datas em datetime para que possam ser divididas em mês e dia

In [9]:
df['periodo___data_de_inicio'] = pd.to_datetime(
    df['periodo___data_de_inicio'], dayfirst=True, errors='coerce'
)

df['periodo___data_de_fim'] = pd.to_datetime(
    df['periodo___data_de_fim'], dayfirst=True, errors='coerce'
)

df['mes_ida'] = df['periodo___data_de_inicio'].dt.month_name()
df['mes_vol'] = df['periodo___data_de_fim'].dt.month_name()

df['dat_ini'] = df['periodo___data_de_inicio'].dt.date
df['dat_fim'] = df['periodo___data_de_fim'].dt.date

Padroniza os valores de viagem_urgente para que possa ser facilmente identificado

In [10]:
df['viagem_urgente'] = (
    df['viagem_urgente']
    .str.upper()
    .replace({'NAO': 'NÃO'})
)

Dropa as linhas que possuem valores nulos e salva quantas foram removidas na variável linhas_valores_invalidos

In [11]:
linhas_valores_invalidos = df[df.isna().any(axis=1)]
total_linhas_valores_invalidos = len(linhas_valores_invalidos)

df = df.dropna(axis=0, how="any")

total_linhas_validas = len(df)

Renomeia as colunas da tabela para que sejam usados os mnemônios 

In [12]:
df = df.rename(columns={
    'identificador_do_processo_de_viagem': 'ide_pro_via',
    'numero_da_proposta_(pcdp)': 'nmr_ppt_pcd',
    'situacao': 'situacao',
    'viagem_urgente': 'via_urg',
    'justificativa_urgencia_viagem': 'jus_urg_via',
    'codigo_do_orgao_superior': 'cod_org_sup',
    'nome_do_orgao_superior': 'nom_org_sup',
    'codigo_orgao_solicitante': 'cod_org_sol',
    'nome_orgao_solicitante': 'nom_org_sol',
    'cpf_viajante': 'cpf_vjt',
    'nome': 'nom_vjt',
    'cargo': 'cargo',
    'funcao': 'funcao',
    'dat_ini': 'dat_ini',
    'dat_fim': 'dat_fim',
    'destinos': 'destinos',
    'motivo': 'motivo',
    'valor_diarias': 'vlr_dia',
    'valor_passagens': 'vlr_psg',
    'valor_devolucao': 'vlr_dvl',
    'valor_outros_gastos': 'vlr_out'
})

Organiza o df para que possa ser salvo de forma correta e padronizada no banco

In [13]:
df = df[[
    'ide_pro_via',
    'nmr_ppt_pcd',
    'situacao',
    'via_urg',
    'jus_urg_via',
    'cod_org_sup',
    'nom_org_sup',
    'cod_org_sol',
    'nom_org_sol',
    'cpf_vjt',
    'nom_vjt',
    'cargo',
    'funcao',
    'dat_ini',
    'dat_fim',
    'destinos',
    'motivo',
    'vlr_dia',
    'vlr_psg',
    'vlr_dvl',
    'vlr_out',
    'tot_gas',
    'mes_ida',
    'mes_vol'
]]


Carrega os dados no banco Postgressql

In [14]:
df.to_sql(
    'viagem',
    engine,
    if_exists='append',
    index=False,
    method='multi'
)

130531

Gera um relatório rápido de transparência para explicar exatamente o que foi feito com os dados para o cliente

In [15]:
print("RELATÓRIO DE CARGA – CAMADA SILVER")
print(f"Total de linhas lidas do RAW: {total_linhas_validas}")
print(f"Linhas descartadas por erro estrutural: {len(linhas_descartadas)}")
print(f"Linhas descartadas por valores inválidos: {total_linhas_valores_invalidos}")
print(f"Linhas carregadas no PostgreSQL: {len(df)}")
print("Carga concluída com sucesso.")

RELATÓRIO DE CARGA – CAMADA SILVER
Total de linhas lidas do RAW: 130531
Linhas descartadas por erro estrutural: 0
Linhas descartadas por valores inválidos: 620526
Linhas carregadas no PostgreSQL: 130531
Carga concluída com sucesso.
